<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Convert PRISM to COG

_by Joe Hamman (CarbonPlan), February 16, 2021_

This notebook converts local copies of PRISM normal rasters to COG

**Inputs:**

- local copies of PRISM data

**Outputs:**

- COGs published to cloud storage

**Notes:**


In [ ]:
import io
import os
import pathlib

from google.cloud import storage
from rasterio.io import MemoryFile
from rio_cogeo.cogeo import cog_translate
from rio_cogeo.profiles import cog_profiles

from carbonplan_data.utils import setup

import xarray as xr
import rioxarray

storage.blob._DEFAULT_CHUNKSIZE = 5 * 1024 * 1024  # 5 MB
storage.blob._MAX_MULTIPART_SIZE = 5 * 1024 * 1024  # 5 MB

In [ ]:
# This is the COG profile:
dst_profile = cog_profiles.get("deflate")

workdir, upload = setup("joe")

## PRISM normals (CONUS and AK)


In [ ]:
sources = [
    "/Users/jhamman/workdir/carbonplan_data_downloads/PRISM_tmean_ak_30yr_normal_800mM1_annual_asc/ak_tmean_1981_2010.14.txt",
    "/Users/jhamman/workdir/carbonplan_data_downloads/PRISM_ppt_ak_30yr_normal_800mM1_annual_asc/ak_ppt_1981_2010.14.txt",
]

crs = "+proj=latlong +ellps=GRS80 +datum=NAD83"

for source in sources:
    target = source.replace("txt", "tif")
    da = xr.open_rasterio(source)
    da = da.rio.set_crs(crs)
    da.rio.to_raster(target)

In [ ]:
sources = [
    "/Users/jhamman/workdir/carbonplan_data_downloads/ppt_normal/PRISM_ppt_30yr_normal_800mM2_annual_bil.bil",
    "/Users/jhamman/workdir/carbonplan_data_downloads/tmean_normal/PRISM_tmean_30yr_normal_800mM2_annual_bil.bil",
    "/Users/jhamman/workdir/carbonplan_data_downloads/PRISM_tmean_ak_30yr_normal_800mM1_annual_asc/ak_tmean_1981_2010.14.tif",
    "/Users/jhamman/workdir/carbonplan_data_downloads/PRISM_ppt_ak_30yr_normal_800mM1_annual_asc/ak_ppt_1981_2010.14.tif",
]

# local target
target = "./raster.tif"

for source in sources:

    for var in ["ppt", "tmean"]:
        if var in source:
            break

    if "ak" in source:
        region = "ak"
    else:
        region = "conus"

    # This is where we'll write the COGs when we're done
    cloud_target = f"raw/prism/normals/{region}/800m/{var}.tif"

    # translate to COG
    cog_translate(source, target, dst_profile)

    # Upload to GCS
    upload(target, cloud_target)

    # Remove temporary file
    os.remove(target)